In [3]:
import cv2
import numpy as np
import face_recognition
import os
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.linalg import norm

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'c985bf2d285249c4aff2c9d8d1250b4f'
secret = '6a1620c44bef4a0aa909ab074f0d0f69'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
path = "images"
images = []
classnames = []
myList = os.listdir(path)
for i in myList:
    curimg = cv2.imread(f'{path}/{i}')
    images.append(curimg)
    classnames.append(os.path.splitext(i)[0])
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [6]:
def findEncodings(images):
    encodelist = []
    for img in images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodelist.append(encode)
    return encodelist

In [7]:
encodelistknown = findEncodings(images)
print(f"{len(encodelistknown)} Images encoded")

4 Images encoded


In [8]:
from tensorflow.keras.models import load_model
model = load_model('emotiondet.h5')

In [9]:
import cv2
from tensorflow.keras.utils import img_to_array
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [10]:
emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [11]:
def recommend_base_mood(mood,name):    
    df = pd.read_csv('songdataset_withmood.csv')
    if mood=='happy':
        df_mood = df[df.mood==mood]
    elif mood=='surprised':
        df_mood = df[df.mood==mood]
    elif (mood=='sad')or(mood=='fear'):
        df_mood = df[df.mood=='sad']
    elif (mood=='angry')or(mood=='disgust'):
        df_mood = df[df.mood=='angry']
    else:
        df_mood = df.sample(n=100)
    df_mood.drop(['mood'],axis = 1)
    if name!='unknown':
        df_mood.to_csv(f'{name}_{mood}.csv')

In [12]:
def emotiondet():    
    cap = cv2.VideoCapture(0)
    while True:
        res, frame = cap.read()
        frames = cv2.resize(frame,(0,0),None,0.25,0.25)
        frames = cv2.cvtColor(frames,cv2.COLOR_BGR2RGB)
        height, width , channel = frame.shape
        gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_haar_cascade.detectMultiScale(gray_image)
        faceloc = face_recognition.face_locations(frames)
        encode = face_recognition.face_encodings(frames,faceloc)
        try:
            for i,j in zip(encode,faceloc):
                matches = face_recognition.compare_faces(encodelistknown,i)
                facedis = face_recognition.face_distance(encodelistknown,i)
                matchindex = np.argmin(facedis)
                if matches[matchindex]:
                    name = classnames[matchindex].upper()
                    y1,x2,y2,x1 = j
                    y1,x2,y2,x1 = (y1)*4,(x2)*4,(y2)*4,(x1)*4
                    roi_gray = gray_image[y1-5:y2+5,x1-5:x2+5]
                    roi_gray=cv2.resize(roi_gray,(48,48))
                    image_pixels = img_to_array(roi_gray)
                    image_pixels = np.expand_dims(image_pixels, axis = 0)
                    image_pixels /= 255
                    predictions = model.predict(image_pixels)
                    max_index = np.argmax(predictions[0])
                    emotion_prediction = emotion_detection[max_index]
                    frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
                    frame = cv2.rectangle(frame,(x1,y2+35),(x2,y2),(0,255,0),cv2.FILLED)
                    frame = cv2.putText(frame,name,(x1,y2+35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                    frame = cv2.putText(frame,emotion_prediction,(x1,y1-10),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

        except:
            pass
        cv2.imshow('webcam',frame)
        key = cv2.waitKey(1)
        if key==27:
            em_for_rec = emotion_prediction
            name_for_rec = name
            break

    cap.release()
    cv2.destroyAllWindows()
    return em_for_rec,name_for_rec

In [13]:
def recommend(n_recs = 5):
    track_url = input("enter the link to the song o spotify")
    track_id = track_url.split("/")[-1].split("?")[0]
    track_features = sp.audio_features(track_id)[0]
    track_moodvec = np.array([track_features['valence'], track_features['energy']])
    df = pd.read_csv('songdataset_withmood.csv')
    df["mood_vec"] = df[["valence", "energy"]].values.tolist()
    df["distances"] = df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
    df_sorted = df.sort_values(by = "distances", ascending = True)
    df_sorted = df_sorted[df_sorted["track_id"] != track_id]
    return df_sorted.iloc[:n_recs]

In [14]:
print("Enter the choice 1/2: \n1) emotion based recommendation\n2)Song based recomendation")
a = int(input("enter the choice:"))
if a==1:
    em_for_rec,name_for_rec = emotiondet()
    recommend_base_mood(em_for_rec,name_for_rec)
elif a==2:
    n = int(input("enter the number of songs to be recommended"))
    df = recommend(n)
    display(df)

Enter the choice 1/2: 
1) emotion based recommendation
2)Song based recomendation
enter the choice:1
1/1 [==============================] - 0s 33ms/step


In [35]:
display(df)

,Unnamed: 0,track_id,artist_name,track_name,popularity,genre,danceability,energy,key,loudness,...,tempo,track_type,uri,track_href,analysis_url,duration_ms,time_signature,mood,mood_vec,distances
389,389,4HBZA5flZLE435QTztThqH,Ariana Grande,Stuck with U (with Justin Bieber),83,"['dance pop', 'pop']",0.597,0.450,8,-6.658,...,178.765,audio_features,spotify:track:4HBZA5flZLE435QTztThqH,https://api.spotify.com/v1/tracks/4HBZA5flZLE4...,https://api.spotify.com/v1/audio-analysis/4HBZ...,228482,3,happy,"[0.537, 0.45]",0.012042
176,176,2Fxmhks0bxGSBdJ92vM42m,Billie Eilish,bad guy,83,"['art pop', 'electropop', 'pop']",0.701,0.425,7,-10.965,...,135.128,audio_features,spotify:track:2Fxmhks0bxGSBdJ92vM42m,https://api.spotify.com/v1/tracks/2Fxmhks0bxGS...,https://api.spotify.com/v1/audio-analysis/2Fxm...,194088,4,sad,"[0.562, 0.425]",0.023345
164,164,4r8JqkhpTb5tzSKKHnVJIJ,Mohit Chauhan,Tujhe Bhula Diya,67,"['desi pop', 'filmi', 'modern bollywood']",0.644,0.423,11,-8.702,...,131.903,audio_features,spotify:track:4r8JqkhpTb5tzSKKHnVJIJ,https://api.spotify.com/v1/tracks/4r8JqkhpTb5t...,https://api.spotify.com/v1/audio-analysis/4r8J...,279429,4,surprised,"[0.509, 0.423]",0.041593
21,21,34gCuhDGsG4bRPIf9bb02f,Ed Sheeran,Thinking out Loud,83,"['pop', 'uk pop']",0.781,0.445,2,-6.061,...,78.998,audio_features,spotify:track:34gCuhDGsG4bRPIf9bb02f,https://api.spotify.com/v1/tracks/34gCuhDGsG4b...,https://api.spotify.com/v1/audio-analysis/34gC...,281560,4,angry,"[0.591, 0.445]",0.045100
132,132,6FZDfxM3a3UCqtzo5pxSLZ,Halsey,Without Me,79,"['dance pop', 'electropop', 'etherpop', 'indie...",0.752,0.488,6,-7.050,...,136.041,audio_features,spotify:track:6FZDfxM3a3UCqtzo5pxSLZ,https://api.spotify.com/v1/tracks/6FZDfxM3a3UC...,https://api.spotify.com/v1/audio-analysis/6FZD...,201661,4,sad,"[0.533, 0.488]",0.047802
327,327,4LO5DjN6Tp3UkUvgESuy2t,Jawad Ahmad,Samjhawan,64,"['classic pakistani pop', 'pakistani pop']",0.650,0.482,4,-8.667,...,106.994,audio_features,spotify:track:4LO5DjN6Tp3UkUvgESuy2t,https://api.spotify.com/v1/tracks/4LO5DjN6Tp3U...,https://api.spotify.com/v1/audio-analysis/4LO5...,269185,4,angry,"[0.517, 0.482]",0.049406
218,218,1iQDltZqI7BXnHrFy4Qo1k,SHAED,Trampoline (with ZAYN),68,"['electropop', 'pop']",0.619,0.459,7,-5.782,...,126.803,audio_features,spotify:track:1iQDltZqI7BXnHrFy4Qo1k,https://api.spotify.com/v1/tracks/1iQDltZqI7BX...,https://api.spotify.com/v1/audio-analysis/1iQD...,184280,4,surprised,"[0.498, 0.459]",0.050922
170,170,2U4tbK3U9DFdZvhLNagF5V,Liam Payne,Strip That Down (feat. Quavo),0,"['dance pop', 'pop', 'post-teen pop', 'uk pop']",0.873,0.495,6,-5.446,...,106.033,audio_features,spotify:track:2U4tbK3U9DFdZvhLNagF5V,https://api.spotify.com/v1/tracks/2U4tbK3U9DFd...,https://api.spotify.com/v1/audio-analysis/2U4t...,202062,4,angry,"[0.546, 0.495]",0.053000
348,348,2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,Monster (Shawn Mendes & Justin Bieber),73,"['canadian pop', 'dance pop', 'pop', 'viral pop']",0.652,0.383,2,-7.076,...,145.765,audio_features,spotify:track:2Z8yfpFX0ZMavHkcIeHiO1,https://api.spotify.com/v1/tracks/2Z8yfpFX0ZMa...,https://api.spotify.com/v1/audio-analysis/2Z8y...,178994,4,sad,"[0.549, 0.383]",0.059076
374,374,4IhKLu7Vk3j2TLmnFPl6To,Alec Benjamin,Water Fountain,79,"['alt z', 'dream smp', 'electropop', 'pop']",0.723,0.504,8,-5.572,...,133.975,audio_features,spotify:track:4IhKLu7Vk3j2TLmnFPl6To,https://api.spotify.com/v1/tracks/4IhKLu7Vk3j2...,https://api.spotify.com/v1/audio-analysis/4IhK...,218893,4,sad,"[0.551, 0.504]",0.062201
